In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import warnings; warnings.simplefilter('ignore')

In [3]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [6]:
from google.colab import drive

In [9]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
md = pd. read_csv('/content/drive/MyDrive/ratings1.csv')
md.head()

,userId,rating,timestamp,movieId,fecha,date,platform
0,1,1.0,1425941529,as680,2015-03-09 22:52:09,1970-01-17,amazon
1,1,4.5,1425942435,ns2186,2015-03-09 23:07:15,1970-01-17,netflix
2,1,5.0,1425941523,hs2381,2015-03-09 22:52:03,1970-01-17,hulu
3,1,5.0,1425941546,ns3663,2015-03-09 22:52:26,1970-01-17,netflix
4,1,5.0,1425941556,as9500,2015-03-09 22:52:36,1970-01-17,amazon


In [30]:
ratings = md[['userId', 'movieId', 'rating']]
ratings.head()

,userId,movieId,rating
0,1,as680,1.0
1,1,ns2186,4.5
2,1,hs2381,5.0
3,1,ns3663,5.0
4,1,as9500,5.0


## Filtración colaborativa

Utilizaremos una técnica llamada **Filtrado colaborativo** para hacer recomendaciones a los espectadores de películas. El filtrado colaborativo se basa en la idea de que los usuarios similares a mí pueden usarse para predecir cuánto me gustará un producto o servicio en particular que esos usuarios han usado/experimentado pero yo no.

No implementaré el filtrado colaborativo desde cero. En su lugar, usaré la biblioteca **Surprise** que usó algoritmos extremadamente poderosos como **Descomposición de valores singulares (SVD)** para minimizar el RMSE (Error cuadrático medio) y brindar excelentes recomendaciones.

In [20]:
reader = Reader()

In [31]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
from surprise.model_selection import KFold
kf = KFold(n_splits=5)
kf.split(data)
#data.split(n_folds=5)

<generator object KFold.split at 0x7f8e0b6d87b0>

In [32]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'],cv=5,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9738  0.9751  0.9714  0.9753  0.9727  0.9737  0.0015  
MAE (testset)     0.7576  0.7581  0.7572  0.7584  0.7566  0.7576  0.0007  
Fit time          3.90    4.24    4.65    3.94    5.12    4.37    0.46    
Test time         0.68    0.92    0.60    0.62    0.93    0.75    0.14    


{'test_rmse': array([0.97384012, 0.97514359, 0.97140224, 0.97525772, 0.97271562]),
 'test_mae': array([0.75764612, 0.75810631, 0.75721812, 0.75841124, 0.75655719]),
 'fit_time': (3.899815082550049,
  4.23884654045105,
  4.646897554397583,
  3.940037250518799,
  5.119748115539551),
 'test_time': (0.6832740306854248,
  0.917586088180542,
  0.6036725044250488,
  0.6207945346832275,
  0.9262785911560059)}

Obtenemos una media de **Root Mean Square Error** de  0.9714, que es más que suficiente para nuestro caso. Entrenemos ahora en nuestro conjunto de datos y lleguemos a las predicciones.

In [33]:
#from surprise import KNNBasic
#data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
svd.fit(trainset)

Elijamos al usuario 5000 y verifiquemos las calificaciones que ha dado.

In [34]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating
0,1,as680,1.0
1,1,ns2186,4.5
2,1,hs2381,5.0
3,1,ns3663,5.0
4,1,as9500,5.0
5,1,as3004,4.0
6,1,ns8282,4.5
7,1,as6112,5.0
8,1,hs327,4.0
9,1,ns1358,4.0


In [35]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=4.135273973639967, details={'was_impossible': False})

Para la película con ID 302, obtenemos una predicción estimada de **4.1352**. Una característica sorprendente de este sistema de recomendación es que no le importa cuál es la película (o qué contiene). Funciona únicamente sobre la base de una ID de película asignada e intenta predecir las calificaciones en función de cómo los otros usuarios han predicho la película.